In [0]:
%sql
-- Create feedback table using SQL
USE CATALOG dev_1899989130012056;
USE SCHEMA ai_agents;

CREATE TABLE IF NOT EXISTS agent_feedback (
    feedback_id STRING,
    question STRING,
    answer STRING,
    rating INT,
    comments STRING,
    timestamp TIMESTAMP
) USING DELTA;

-- Create metrics table
CREATE TABLE IF NOT EXISTS agent_metrics (
    metric_date DATE,
    total_queries INT,
    avg_response_time FLOAT,
    satisfaction_rate FLOAT,
    top_topics ARRAY<STRING>
) USING DELTA;

In [0]:
import uuid

def log_feedback(question, answer, rating, comments=""):
    """Log user feedback for continuous improvement"""
    feedback_data = [{
        "feedback_id": str(uuid.uuid4()),
        "question": question,
        "answer": answer,
        "rating": rating,
        "comments": comments,
        "timestamp": datetime.now()
    }]
    
    spark.createDataFrame(feedback_data).write.mode("append").saveAsTable(
        f"{catalog_name}.{schema_name}.agent_feedback"
    )

In [0]:
# Prepare training data from feedback
def prepare_training_data():
    """Prepare high-quality Q&A pairs for fine-tuning"""
    
    # Get positive feedback examples
    good_examples = spark.sql(f"""
        SELECT question, answer 
        FROM {catalog_name}.{schema_name}.agent_feedback
        WHERE rating >= 4
    """).toPandas()
    
    return good_examples

# Note: Actual fine-tuning would require additional setup
# This is a placeholder for the fine-tuning process